In [85]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import math
import urllib.request
import os

import time
import warnings
warnings.filterwarnings("ignore")

In [7]:
options = Options()
options.headless = True
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--blink-settings=imagesEnabled=false')

# All Tickers

In [8]:
r = requests.get('http://www.tsetmc.com/Loader.aspx?ParTree=111C1417', headers={"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}, allow_redirects=False)
soup = BeautifulSoup(r.text, 'html.parser')
tables = soup.findAll('tr')
l = []
for tr in tables:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
all_tickers = pd.DataFrame(l).iloc[2:, 6:].reset_index().drop('index', axis = 1)
all_tickers.columns = ['Ticker', 'Name']
all_tickers = all_tickers[[not('ح .' in x or 'ح . ' in x or 'ح.' in x or 'اختيار' in x) for x in all_tickers['Name']]]
all_tickers

,Ticker,Name
0,آباد,توريستي ورفاهي آبادگران ايران
1,آبادا,توليد نيروي برق آبادان
3,آپ,آسان پرداخت پرشين
4,آسيا,بيمه آسيا
5,آكنتور,كنتورسازي‌ايران‌
...,...,...
886,وملي,گروه‌ صنعتي‌ ملي‌ (هلدينگ‌
888,ونفت,سرمايه‌گذاري‌ صنعت‌ نفت‌
890,ونوين,بانك‌اقتصادنوين‌
892,ونيرو,سرمايه‌گذاري‌نيرو


# Dividend Data

In [41]:
def DVDND_SCRAP_EX(DVDND, ticker, year, check_list):
    codal_url = 'https://codal.ir'
    url_head = 'https://codal.ir/ReportList.aspx?search&Symbol='
    url_tail_ = '&LetterType=21&Isic=272006&AuditorRef=-1&Audited&NotAudited&IsNotAudited=false&Childs&Mains&Publisher=false&CompanyState=0&Length=12&Category=6&CompanyType=-1&Consolidatable&NotConsolidatable&FromDate=' + str(year) +'%2F01%2F01&ToDate=' + str(year+1) + '%2F01%2F01'
    url = url_head + ticker + url_tail_
    driver2 = webdriver.Chrome(options = options, chrome_options = chrome_options)
    driver2.get(url)
    try:
        WebDriverWait(driver2, 2).until(EC.presence_of_element_located((By.CLASS_NAME, 'letter-title')))
    except:
        1
    page_source = driver2.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    letters = soup.find_all('a', class_ = 'letter-title')
    dates = soup.find_all('td', {'data-heading': 'زمان انتشار'})
    for j, letter in enumerate(letters):
        year = int(dates[j].find('span').decode_contents().split('/')[0])
        if not(math.isnan(DVDND.loc[year, ticker])):
            continue
        table_href = codal_url + letter.attrs['href']
        table_page = requests.get(table_href, headers={"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}, allow_redirects=False)
        table_soup = BeautifulSoup(table_page.text, 'html.parser')
        try:
            table_div = table_soup.find('div', {'id' : 'divAssemblyProportionedRetainedEarning'}).find_all('td')
            for i, td in enumerate(table_div):
                if 'سود نقدی هر سهم' in td.decode_contents():
                    dividend_str = table_div[i+1].find('span').decode_contents()
                    if ',' in dividend_str:
                        dividend_str_splitted = table_div[i+1].find('span').decode_contents().split(',')
                        dividend_str = dividend_str_splitted[0] + dividend_str_splitted[1]
                    dividend = int(dividend_str)
            
            DVDND.loc[year, ticker] = dividend
        except:
            1
    if math.isnan(DVDND.loc[year, ticker]):
        check = ticker + ': ' + str(year)
        check_list.append(check)
    driver2.quit()    

In [42]:
def DVDND_SCRAP(tickers, DVDND, pages = '1'):
    check_list = []
    for t, ticker in  enumerate(tickers):
        codal_url = 'https://codal.ir'
        url_head = 'https://codal.ir/ReportList.aspx?search&Symbol='
        url_tail = '&LetterType=20&Isic=272006&AuditorRef=-1&Audited&NotAudited&IsNotAudited=false&Childs&Mains&Publisher=false&CompanyState=0&Length=12&Category=6&CompanyType=-1&Consolidatable&NotConsolidatable&PageNumber='
        url = url_head + ticker + url_tail + pages
        driver = webdriver.Chrome(options = options, chrome_options = chrome_options)
        driver.get(url)
        try:
            WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.CLASS_NAME, 'letter-title')))
        except:
            1
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        letters = soup.find_all('a', class_ = 'letter-title')
        dates = soup.find_all('td', {'data-heading': 'زمان انتشار'})
        for j, letter in enumerate(letters):
            year = int(dates[j].find('span').decode_contents().split('/')[0])
            if not(math.isnan(DVDND.loc[year, ticker])):
                continue
            table_href = codal_url + letter.attrs['href']
            table_page = requests.get(table_href, headers={"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}, allow_redirects=False)
            table_soup = BeautifulSoup(table_page.text, 'html.parser')
            try:
                table_div = table_soup.find('div', {'id' : 'divAssemblyProportionedRetainedEarning'}).find_all('td')
                for i, td in enumerate(table_div):
                    if 'سود نقدی هر سهم' in td.decode_contents():
                        dividend_str = table_div[i+1].find('span').decode_contents()
                        if ',' in dividend_str:
                            dividend_str_splitted = table_div[i+1].find('span').decode_contents().split(',')
                            dividend_str = dividend_str_splitted[0] + dividend_str_splitted[1]
                        dividend = int(dividend_str)
                
                DVDND.loc[year, ticker] = dividend
            except:
                DVDND_SCRAP_EX(DVDND, ticker, year, check_list)
        
        try:
            result_count = int(driver.find_element(By.CSS_SELECTOR, "div.result-count span").text) #int(soup.find('div', {'class' : 'result-count'}).find('span').decode_contents())
            page_numbers = int(np.ceil(result_count / 20))
        except:
            page_numbers = 1
        if page_numbers > 1 and pages == '1':
            for page in range(2, page_numbers+1):
                DVDND_SCRAP([ticker], DVDND, pages = str(page))
        print(t)
        

    return DVDND, check_list

In [ ]:
DVDND = pd.DataFrame(index = range(1388, 1403), columns = all_tickers['Ticker'])
DVDND, check_list = DVDND_SCRAP(all_tickers['Ticker'], DVDND);

In [64]:
DVDND

Ticker,آباد,آبادا,آپ,آسيا,آكنتور,آلومينا,اپال,اتكام,اجداد,اخابر,...,وليز,ومدير,ومعادن,ومعين,وملت,وملي,ونفت,ونوين,ونيرو,ونيكي
1388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,100
1389,50,NaN,NaN,NaN,1000,NaN,NaN,NaN,NaN,NaN,...,400,NaN,450,NaN,2,0,27,450,12,150
1390,169,NaN,NaN,10,11,NaN,NaN,NaN,1250,NaN,...,500,NaN,500,NaN,0,0,9,465,100,150
1391,175,NaN,NaN,NaN,500,NaN,NaN,NaN,NaN,420,...,383,NaN,250,NaN,NaN,0,NaN,350,250,NaN
1392,180,NaN,NaN,NaN,450,NaN,NaN,NaN,NaN,425,...,363,NaN,1000,NaN,0,33,0,380,350,330
1393,200,NaN,NaN,NaN,550,NaN,NaN,NaN,NaN,435,...,350,NaN,370,NaN,0,490,100,330,100,330
1394,230,NaN,NaN,10,NaN,NaN,NaN,200,NaN,400,...,350,NaN,450,16,0,550,122,220,0,210
1395,10,NaN,0,0,NaN,NaN,NaN,200,NaN,450,...,0,NaN,150,NaN,0,300,200,0,50,NaN
1396,0,NaN,1000,70,NaN,NaN,NaN,210,NaN,333,...,0,NaN,50,NaN,0,2,120,0,250,50
1397,5,NaN,590,100,NaN,NaN,NaN,300,NaN,400,...,0,NaN,200,NaN,0,2,150,0,NaN,125


In [65]:
DVDND.to_csv('Dividends1.csv')